In [1]:
import re
import pandas as pd
from bs4 import BeautifulSoup
import requests

# 일주일 간격으로 크롤링
day_start = ["2019.12.01", "2019.12.08", "2019.12.15", "2019.12.22", "2019.12.29", "2020.01.05", "2020.01.12", "2020.01.19", "2020.01.26", "2020.02.02", "2020.02.09", "2020.02.16", "2020.02.23", "2020.03.01", "2020.03.08", "2020.03.15", "2020.03.22", "2020.03.29", "2020.04.05", "2020.04.12", "2020.04.19", "2020.04.26", "2020.05.03", "2020.05.10", "2020.05.17", "2020.05.24", "2020.05.31", "2020.06.07", "2020.06.14", "2020.06.21", "2020.06.28", "2020.07.05", "2020.07.12", "2020.07.19", "2020.07.26", "2020.08.02", "2020.08.09", "2020.08.16", "2020.08.23", "2020.08.30", "2020.09.06", "2020.09.13", "2020.09.20", "2020.09.27", "2020.10.04", "2020.10.11", "2020.10.18", "2020.10.25"]
day_end = ["2019.12.07", "2019.12.14", "2019.12.21", "2019.12.28", "2020.01.04", "2020.01.11", "2020.01.18", "2020.01.25", "2020.02.01", "2020.02.08", "2020.02.15", "2020.02.22", "2020.02.29", "2020.03.07", "2020.03.14", "2020.03.21", "2020.03.28", "2020.04.04", "2020.04.11", "2020.04.18", "2020.04.25", "2020.05.02", "2020.05.09", "2020.05.16", "2020.05.23", "2020.05.30", "2020.06.06", "2020.06.13", "2020.06.20", "2020.06.27", "2020.07.04", "2020.07.11", "2020.07.18", "2020.07.25", "2020.08.01", "2020.08.08", "2020.08.15", "2020.08.22", "2020.08.29", "2020.09.05", "2020.09.12", "2020.09.19", "2020.09.26", "2020.10.03", "2020.10.10", "2020.10.17", "2020.10.24", "2020.10.31"]
headers = {'User-Agent': 'Mozilla/5.0'} 

# 네이버 기사 포맷으로 제공되지만 에러가 발생한 URL
error_url = []

for i in range(len(day_start)):
    covid_news_list = []
    start = day_start[i]
    end = day_end[i]
    for j in range(401): # 기간검색 최대 검색량 4000개 
        try: # 만약 페이지를 넘어가면 오류를 뱉음
            news_url = f'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EC%BD%94%EB%A1%9C%EB%82%98%2C%20%ED%95%AD%EA%B3%B5&sort=1&photo=0&field=0&pd=3&ds={start}&de={end}&start={j*10+1}'
            req = requests.get(news_url)
            soup = BeautifulSoup(req.text, 'html.parser')

            table = soup.find('ul',{'class' : 'list_news'})
            li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
            area_list = [li.find('div', {'class' : 'news_area'}) for li in li_list]
            info_list = [info.find('div', {'class' : 'news_info'}) for info in area_list]
            naver_news = [naver.find('a', {'href': re.compile('https://news.naver.com.*')}) for naver in info_list]

            for news in naver_news:
                if news != None:
                    covid_news_list.append(news)
                else:
                    pass
        except: # 오류가 나면 안긴 반복문 종료
            print(f'{start} 오류 발생, 제공 기사수 4000개 이하')
            break
    
    # url을 포함한 a태크 수집
    covid_news_list_url = []

    # a태그에서 href 즉, 기사 url 수집
    for i in covid_news_list:
        covid_news_list_url.append(i.get('href'))

    print(f"url 수집 완료({len(covid_news_list_url)}개), 기사 추출 시작")

    # 뉴스 본문 모으기
    news_list = []
    for url in covid_news_list_url:
        try:
            news_dict = {}
            req = requests.get(url, headers = headers)
            soup = BeautifulSoup(req.text, 'html.parser')

            table = soup.find('div',{'id' : 'main_content'})
            title = table.find('h3', {'id':'articleTitle'})
            body = table.find('div', {'id':'articleBodyContents'})

            # 뉴스 제목과 본문 수집
            news_dict['title'] = title.get_text()
            news_dict['body'] = body.get_text()
            news_list.append(news_dict)
        except:
            # 네이버 기사 포맷으로 제공되지만 에러가 발생한 URL
            print(f'오류 발생, {start}_{end}의 {i}번째')
            error_url.append(url)
            pass

    article_list = pd.DataFrame(news_list)
    article_list.to_csv(f'naver_article/article_list_{start}_{end}.csv', encoding='utf-8-sig')
    print(f'naver_article/article_list_{start}_{end}: 완료')
    print(f'기사 개수: {len(news_list)}')


2019.12.01 오류 발생, 제공 기사수 4000개 이하
url 수집 완료(3개), 기사 추출 시작
naver_article/article_list_2019.12.01_2019.12.07: 완료
기사 개수: 3
2019.12.08 오류 발생, 제공 기사수 4000개 이하
url 수집 완료(2개), 기사 추출 시작
naver_article/article_list_2019.12.08_2019.12.14: 완료
기사 개수: 2
2019.12.15 오류 발생, 제공 기사수 4000개 이하
url 수집 완료(0개), 기사 추출 시작
naver_article/article_list_2019.12.15_2019.12.21: 완료
기사 개수: 0
2019.12.22 오류 발생, 제공 기사수 4000개 이하
url 수집 완료(0개), 기사 추출 시작
naver_article/article_list_2019.12.22_2019.12.28: 완료
기사 개수: 0
2019.12.29 오류 발생, 제공 기사수 4000개 이하
url 수집 완료(1개), 기사 추출 시작
naver_article/article_list_2019.12.29_2020.01.04: 완료
기사 개수: 1
2020.01.05 오류 발생, 제공 기사수 4000개 이하
url 수집 완료(23개), 기사 추출 시작
naver_article/article_list_2020.01.05_2020.01.11: 완료
기사 개수: 23
2020.01.12 오류 발생, 제공 기사수 4000개 이하
url 수집 완료(40개), 기사 추출 시작
naver_article/article_list_2020.01.12_2020.01.18: 완료
기사 개수: 40
2020.01.19 오류 발생, 제공 기사수 4000개 이하
url 수집 완료(1296개), 기사 추출 시작
naver_article/article_list_2020.01.19_2020.01.25: 완료
기사 개수: 1296
url 수집 완료(2371개), 기사 추출 시작
오류 

In [1]:
import re
import pandas as pd
from bs4 import BeautifulSoup
import requests

# 일주일 간격으로 크롤링
day_start = ["2019.06.30","2019.07.07","2019.07.14","2019.07.21","2019.07.28","2019.08.04","2019.08.11","2019.08.18","2019.08.25","2019.09.01","2019.09.08","2019.09.15","2019.09.22","2019.09.29","2019.10.06","2019.10.13","2019.10.20","2019.10.27","2019.11.03","2019.11.10","2019.11.17","2019.11.24"]
day_end = ["2019.07.06","2019.07.13","2019.07.20","2019.07.27","2019.08.03","2019.08.10","2019.08.17","2019.08.24","2019.08.31","2019.09.07","2019.09.14","2019.09.21","2019.09.28","2019.10.05","2019.10.12","2019.10.19","2019.10.26","2019.11.02","2019.11.09","2019.11.16","2019.11.23","2019.11.30"]
headers = {'User-Agent': 'Mozilla/5.0'} 

# 네이버 기사 포맷으로 제공되지만 에러가 발생한 URL
error_url = []
for i in range(len(day_start)):
    covid_news_list = []
    start = day_start[i]
    end = day_end[i]
    for j in range(401): # 기간검색 최대 검색량 4000개 
        try: # 만약 페이지를 넘어가면 오류를 뱉음
            news_url = f'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%ED%95%AD%EA%B3%B5%EC%82%AC&sort=1&photo=0&field=0&pd=3&ds={start}&de={end}&start={j*10+1}'
            req = requests.get(news_url)
            soup = BeautifulSoup(req.text, 'html.parser')

            table = soup.find('ul',{'class' : 'list_news'})
            li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
            area_list = [li.find('div', {'class' : 'news_area'}) for li in li_list]
            info_list = [info.find('div', {'class' : 'news_info'}) for info in area_list]
            naver_news = [naver.find('a', {'href': re.compile('https://news.naver.com.*')}) for naver in info_list]

            for news in naver_news:
                if news != None:
                    covid_news_list.append(news)
                else:
                    pass
        except: # 오류가 나면 안긴 반복문 종료
            print(f'{start} 오류 발생, 제공 기사수 4000개 이하')
            break
    
    # url을 포함한 a태크 수집
    covid_news_list_url = []

    # a태그에서 href 즉, 기사 url 수집
    for i in covid_news_list:
        covid_news_list_url.append(i.get('href'))

    print(f"url 수집 완료({len(covid_news_list_url)}개), 기사 추출 시작")

    # 뉴스 본문 모으기
    news_list = []
    for url in covid_news_list_url:
        try:
            news_dict = {}
            req = requests.get(url, headers = headers)
            soup = BeautifulSoup(req.text, 'html.parser')

            table = soup.find('div',{'id' : 'main_content'})
            title = table.find('h3', {'id':'articleTitle'})
            body = table.find('div', {'id':'articleBodyContents'})

            # 뉴스 제목과 본문 수집
            news_dict['title'] = title.get_text()
            news_dict['body'] = body.get_text()
            news_list.append(news_dict)
        except:
            # 네이버 기사 포맷으로 제공되지만 에러가 발생한 URL
            print(f'오류 발생, {start}_{end}의 {i}번째')
            error_url.append(url)
            pass

    article_list = pd.DataFrame(news_list)
    article_list.to_csv(f'naver_article/article_list_{start}_{end}.csv', encoding='utf-8-sig')
    print(f'naver_article/article_list_{start}_{end}: 완료')
    print(f'기사 개수: {len(news_list)}')


2019.06.30 오류 발생, 제공 기사수 4000개 이하
url 수집 완료(342개), 기사 추출 시작
오류 발생, 2019.06.30_2019.07.06의 <a class="info" href="https://news.naver.com/main/read.naver?mode=LSD&amp;mid=sec&amp;sid1=101&amp;oid=001&amp;aid=0010922236" onclick="return goOtherCR(this, 'a=nws*e.nav&amp;r=1017&amp;i=880000D8_000000000000000010922236&amp;u='+urlencode(this.href));" target="_blank">네이버뉴스</a>번째
오류 발생, 2019.06.30_2019.07.06의 <a class="info" href="https://news.naver.com/main/read.naver?mode=LSD&amp;mid=sec&amp;sid1=101&amp;oid=001&amp;aid=0010922236" onclick="return goOtherCR(this, 'a=nws*e.nav&amp;r=1017&amp;i=880000D8_000000000000000010922236&amp;u='+urlencode(this.href));" target="_blank">네이버뉴스</a>번째
오류 발생, 2019.06.30_2019.07.06의 <a class="info" href="https://news.naver.com/main/read.naver?mode=LSD&amp;mid=sec&amp;sid1=101&amp;oid=001&amp;aid=0010922236" onclick="return goOtherCR(this, 'a=nws*e.nav&amp;r=1017&amp;i=880000D8_000000000000000010922236&amp;u='+urlencode(this.href));" target="_blank">네이버뉴스</a>번째
n

In [2]:
news_url

'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%ED%95%AD%EA%B3%B5%EC%82%AC&sort=1&photo=0&field=0&pd=3&ds=2019.11.24&de=2019.11.30&start=831'

In [3]:
import re
import pandas as pd
from bs4 import BeautifulSoup
import requests

# 일주일 간격으로 크롤링
day_start = ["2019.12.01", "2019.12.08", "2019.12.15", "2019.12.22", "2019.12.29", "2020.01.05", "2020.01.12", "2020.01.19", "2020.01.26", "2020.02.02", "2020.02.09", "2020.02.16", "2020.02.23", "2020.03.01", "2020.03.08", "2020.03.15", "2020.03.22", "2020.03.29", "2020.04.05", "2020.04.12", "2020.04.19", "2020.04.26", "2020.05.03", "2020.05.10", "2020.05.17", "2020.05.24", "2020.05.31", "2020.06.07", "2020.06.14", "2020.06.21", "2020.06.28", "2020.07.05", "2020.07.12", "2020.07.19", "2020.07.26", "2020.08.02", "2020.08.09", "2020.08.16", "2020.08.23", "2020.08.30", "2020.09.06", "2020.09.13", "2020.09.20", "2020.09.27", "2020.10.04", "2020.10.11", "2020.10.18", "2020.10.25"]
day_end = ["2019.12.07", "2019.12.14", "2019.12.21", "2019.12.28", "2020.01.04", "2020.01.11", "2020.01.18", "2020.01.25", "2020.02.01", "2020.02.08", "2020.02.15", "2020.02.22", "2020.02.29", "2020.03.07", "2020.03.14", "2020.03.21", "2020.03.28", "2020.04.04", "2020.04.11", "2020.04.18", "2020.04.25", "2020.05.02", "2020.05.09", "2020.05.16", "2020.05.23", "2020.05.30", "2020.06.06", "2020.06.13", "2020.06.20", "2020.06.27", "2020.07.04", "2020.07.11", "2020.07.18", "2020.07.25", "2020.08.01", "2020.08.08", "2020.08.15", "2020.08.22", "2020.08.29", "2020.09.05", "2020.09.12", "2020.09.19", "2020.09.26", "2020.10.03", "2020.10.10", "2020.10.17", "2020.10.24", "2020.10.31"]
headers = {'User-Agent': 'Mozilla/5.0'} 

# 네이버 기사 포맷으로 제공되지만 에러가 발생한 URL
error_url = []

for i in range(len(day_start)):
    covid_news_list = []
    start = day_start[i]
    end = day_end[i]
    for j in range(401): # 기간검색 최대 검색량 4000개 
        try: # 만약 페이지를 넘어가면 오류를 뱉음
            news_url = f'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%ED%95%AD%EA%B3%B5%EC%82%AC%2C+%EC%BD%94%EB%A1%9C%EB%82%98&sort=1&photo=0&field=0&pd=3&ds={start}&de={end}&start={j*10+1}'
            req = requests.get(news_url)
            soup = BeautifulSoup(req.text, 'html.parser')

            table = soup.find('ul',{'class' : 'list_news'})
            li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
            area_list = [li.find('div', {'class' : 'news_area'}) for li in li_list]
            info_list = [info.find('div', {'class' : 'news_info'}) for info in area_list]
            naver_news = [naver.find('a', {'href': re.compile('https://news.naver.com.*')}) for naver in info_list]

            for news in naver_news:
                if news != None:
                    covid_news_list.append(news)
                else:
                    pass
        except: # 오류가 나면 안긴 반복문 종료
            print(f'{start} 오류 발생, 제공 기사수 4000개 이하')
            break
    
    # url을 포함한 a태크 수집
    covid_news_list_url = []

    # a태그에서 href 즉, 기사 url 수집
    for i in covid_news_list:
        covid_news_list_url.append(i.get('href'))

    print(f"url 수집 완료({len(covid_news_list_url)}개), 기사 추출 시작")

    # 뉴스 본문 모으기
    news_list = []
    for url in covid_news_list_url:
        try:
            news_dict = {}
            req = requests.get(url, headers = headers)
            soup = BeautifulSoup(req.text, 'html.parser')

            table = soup.find('div',{'id' : 'main_content'})
            title = table.find('h3', {'id':'articleTitle'})
            body = table.find('div', {'id':'articleBodyContents'})

            # 뉴스 제목과 본문 수집
            news_dict['title'] = title.get_text()
            news_dict['body'] = body.get_text()
            news_list.append(news_dict)
        except:
            # 네이버 기사 포맷으로 제공되지만 에러가 발생한 URL
            print(f'오류 발생, {start}_{end}의 {i}번째')
            error_url.append(url)
            pass

    article_list = pd.DataFrame(news_list)
    article_list.to_csv(f'naver_article/article_list_{start}_{end}.csv', encoding='utf-8-sig')
    print(f'naver_article/article_list_{start}_{end}: 완료')
    print(f'기사 개수: {len(news_list)}')

2019.12.01 오류 발생, 제공 기사수 4000개 이하
url 수집 완료(0개), 기사 추출 시작
naver_article/article_list_2019.12.01_2019.12.07: 완료
기사 개수: 0
2019.12.08 오류 발생, 제공 기사수 4000개 이하
url 수집 완료(0개), 기사 추출 시작
naver_article/article_list_2019.12.08_2019.12.14: 완료
기사 개수: 0
2019.12.15 오류 발생, 제공 기사수 4000개 이하
url 수집 완료(0개), 기사 추출 시작
naver_article/article_list_2019.12.15_2019.12.21: 완료
기사 개수: 0
2019.12.22 오류 발생, 제공 기사수 4000개 이하
url 수집 완료(0개), 기사 추출 시작
naver_article/article_list_2019.12.22_2019.12.28: 완료
기사 개수: 0
2019.12.29 오류 발생, 제공 기사수 4000개 이하
url 수집 완료(0개), 기사 추출 시작
naver_article/article_list_2019.12.29_2020.01.04: 완료
기사 개수: 0
2020.01.05 오류 발생, 제공 기사수 4000개 이하
url 수집 완료(1개), 기사 추출 시작
naver_article/article_list_2020.01.05_2020.01.11: 완료
기사 개수: 1
2020.01.12 오류 발생, 제공 기사수 4000개 이하
url 수집 완료(2개), 기사 추출 시작
naver_article/article_list_2020.01.12_2020.01.18: 완료
기사 개수: 2
2020.01.19 오류 발생, 제공 기사수 4000개 이하
url 수집 완료(152개), 기사 추출 시작
naver_article/article_list_2020.01.19_2020.01.25: 완료
기사 개수: 152
2020.01.26 오류 발생, 제공 기사수 4000개 이하
ur